In [1]:
import pandas as pd
import sqlalchemy as sa

from config import oltp_conn_string, warehouse_conn_string, oltp_tables, dimension_columns, ddl_statements

In [2]:
conn_oltp = sa.create_engine(oltp_conn_string)
conn_dwh = sa.create_engine(warehouse_conn_string)

In [ ]:
engine = sa.create_engine(oltp_conn_string)

conn = engine.connect()
sql = sa.text("CREATE TABLE IF NOT EXISTS test1 (name VARCHAR(255))")
result = conn.execute(sql)
conn.commit()

In [3]:
engine = sa.create_engine(oltp_conn_string)

conn = engine.connect()
sql = sa.text("SELECT * FROM tb_orders")
result = conn.execute(sql)
conn.commit()
result.fetchall()

[(1110001, datetime.date(2022, 1, 20), 100101, 1202, 600002001, 250000, 15000, 41000101, 230000, 800010001),
 (1110002, datetime.date(2022, 1, 29), 100102, 1202, 600002001, 620000, 40000, 41000102, 575000, 800010002),
 (1110003, datetime.date(2022, 2, 13), 100103, 1204, 600002001, 6000000, 1000000, 41000103, 4995000, 800010003),
 (1110004, datetime.date(2022, 4, 28), 100104, 1203, 600002001, 3150000, 45000, 41000105, 3105000, 800010004),
 (1110005, datetime.date(2022, 5, 21), 100105, 1202, 600002002, 4000000, 1000000, 41000105, 2999500, 800010001),
 (1110006, datetime.date(2022, 6, 3), 100103, 1204, 600002003, 870000, 25000, None, 845000, 800010002),
 (1110007, datetime.date(2022, 6, 15), 100106, 1202, 600002001, 2000000, 0, None, 2000000, 800010002),
 (1110008, datetime.date(2022, 6, 23), 100104, 1204, 600002003, 1050000, 45000, None, 1005000, 800010004),
 (1110009, datetime.date(2022, 7, 1), 100110, 1202, 600002001, 550000, 15000, None, 535000, 800010001),
 (1110010, datetime.date(20


    "insert_dim_sales": """
        TRUNCATE TABLE dim_sales;
        INSERT INTO dim_sales (order_id, order_date, user_id, user_name, payment_type, shipper_name, order_price, order_discount, voucher_name, order_total)
        SELECT
            fo.order_id,
            fo.order_date,
            fo.user_id,
            du.user_first_name || ' ' || du.user_last_name,
            dp.payment_name,
            ds.shipper_name,
            fo.order_price,
            fo.order_discount,
            dv.voucher_name,
            fo.order_total
        FROM fact_orders fo
        INNER JOIN dim_user du ON fo.user_id = du.user_id
        INNER JOIN dim_payment dp ON fo.payment_id = dp.payment_id
        INNER JOIN dim_shipper ds ON fo.shipper_id = ds.shipper_id
        INNER JOIN dim_voucher dv ON fo.voucher_id = dv.voucher_id;"""